In [1]:
import os
from PIL import Image
import numpy as np
import torch
from transformers import ViTFeatureExtractor, ViTModel
from tqdm import tqdm

/opt/anaconda3/envs/SMCA/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
mfcc_path = '../misc/audio_fe/mfcc'
output_path = '../misc/audio_fe/'

mfcc_output_path = os.path.join(output_path, 'mfcc_extracted')

In [5]:
def verify_paths(paths):
    for path in paths:
        if not os.path.exists(path):
            raise FileNotFoundError(f"The path {path} does not exist or is not accessible.")

verify_paths([mfcc_path, output_path])

In [6]:
os.makedirs(mfcc_output_path, exist_ok=True)

In [7]:
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

/opt/anaconda3/envs/SMCA/lib/python3.12/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [8]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)
model.eval()

ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): ViTPatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0-11): 12 x ViTLayer(
        (attention): ViTSdpaAttention(
          (attention): ViTSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUAct

In [9]:
def process_image(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=image, return_tensors="pt")
    inputs = inputs.to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    
    last_token_features = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
    print(f"Processed {image_path} with feature shape: {last_token_features.shape}")
    return last_token_features

In [10]:
def extract_features(input_path, output_path, feature_suffix=""):
    image_files = [f for f in os.listdir(input_path) if f.endswith(".png") or f.endswith(".jpg")]
    
    for filename in tqdm(image_files):
        image_path = os.path.join(input_path, filename)
        feature_vector = process_image(image_path)
        base_filename = os.path.splitext(filename)[0]
        output_file = os.path.join(output_path, f"feature_{base_filename}{feature_suffix}.npy")

        np.save(output_file, feature_vector)


In [11]:
extract_features(mfcc_path, mfcc_output_path)

  0%|          | 3/1356 [00:02<17:23,  1.30it/s]  

Processed ../misc/audio_fe/mfcc/tt0408524_mfcc.png with feature shape: (768,)
Processed ../misc/audio_fe/mfcc/tt0439289_mfcc.png with feature shape: (768,)
Processed ../misc/audio_fe/mfcc/tt0251127_mfcc.png with feature shape: (768,)
Processed ../misc/audio_fe/mfcc/tt0804505_mfcc.png with feature shape: (768,)
Processed ../misc/audio_fe/mfcc/tt0443559_mfcc.png with feature shape: (768,)


  1%|          | 9/1356 [00:03<04:22,  5.13it/s]

Processed ../misc/audio_fe/mfcc/tt0092074_mfcc.png with feature shape: (768,)
Processed ../misc/audio_fe/mfcc/tt0119528_mfcc.png with feature shape: (768,)
Processed ../misc/audio_fe/mfcc/tt1136683_mfcc.png with feature shape: (768,)
Processed ../misc/audio_fe/mfcc/tt0286106_mfcc.png with feature shape: (768,)
Processed ../misc/audio_fe/mfcc/tt0800039_mfcc.png with feature shape: (768,)


  1%|          | 12/1356 [00:03<03:09,  7.11it/s]

...

Processed ../misc/audio_fe/mfcc/tt1176251_mfcc.png with feature shape: (768,)
Processed ../misc/audio_fe/mfcc/tt0120619_mfcc.png with feature shape: (768,)
Processed ../misc/audio_fe/mfcc/tt0896798_mfcc.png with feature shape: (768,)


100%|█████████▉| 1352/1356 [02:24<00:00, 13.93it/s]

Processed ../misc/audio_fe/mfcc/tt0381849_mfcc.png with feature shape: (768,)
Processed ../misc/audio_fe/mfcc/tt1567233_mfcc.png with feature shape: (768,)
Processed ../misc/audio_fe/mfcc/tt0388500_mfcc.png with feature shape: (768,)


100%|██████████| 1356/1356 [02:25<00:00,  9.34it/s]

Processed ../misc/audio_fe/mfcc/tt0299930_mfcc.png with feature shape: (768,)
Processed ../misc/audio_fe/mfcc/tt0119906_mfcc.png with feature shape: (768,)
Processed ../misc/audio_fe/mfcc/tt3526408_mfcc.png with feature shape: (768,)
